# Inference

In [ ]:
#!pip install numpy
from pathlib import Path

from TPTBox import NII

from inference_help import upscale_nii


## Superresolution


You can download a pretrained model here:
[SR_axial_to_sag](https://studentpartnersde-my.sharepoint.com/:f:/g/personal/robert_graf_studentpartners_de/ErqaSmNSB8xMub9xe6R9U3wBiNIPFy37Kwb5z5fTh3pp2g?e=BtzNCI)

Note: As descried in the paper we only improve the sagittal resolution. A second phase would be needed to fix the artifact in the other regions.

In [ ]:
default_checkpoint = "lightning_logs_dae/my_dataset/version_0/checkpoints/last.ckpt"
if Path("lightning_logs_dae/pretrained_ax2sag/version_0/checkpoints/last.ckpt").exists():
    
    default_checkpoint="lightning_logs_dae/pretrained_ax2sag/version_0/checkpoints/last.ckpt"

#lr_img = NII.load('path/to/nii.nii.gz',False)
lr_img = NII.load("/media/data/robert/datasets/dataset-neuropoly-test/rawdata/sub-m073580/ses-20130313/anat/sub-m073580_ses-20130313_acq-ax_chunk-1_T2w.nii.gz",False)
out_path =Path("sub-x_acq-iso_T2w.nii.gz")
hr_img = upscale_nii(lr_img,out_path,batch_size=32,checkpoint_sag=default_checkpoint,override_upscale=True)

## Segmentation
Install the segmentation from 
https://github.com/Hendrik-code/spineps

and follow the instruction.

In [ ]:

#from mri_segmentor import get_model_spine, get_model_vert

#def segment(
#    nii: NII,
#    raw="rawdata",
#    der="derivatives",
#    override_upscale=False,
#    batch_size=32,
#    sort=True,
#    device=torch.device("cuda:0"),  # noqa: B008
#    resample_back=True,
#    sag_only=False,
#):
#    # INPUT
#    in_ds = Path(in_ds)
#    head_logger = No_Logger()  # (in_ds, log_filename="source-convert-to-unet-train", default_verbose=True)#

#    block = ""  # put i.e. 101 in here for block
#    parent_raw = str(Path(raw).joinpath(str(block)))
#    parent_der = str(Path(der).joinpath(str(block)))
#    from mri_segmentor import get_model_spine, get_model_vert#

#    # check available models
#    # modelid2folder_subreg, modelid2folder_vert = check_available_models(model_dir, verbose=True)
#    model_subreg = get_model_spine("T2w_Segmentor").load()
#    try:
#        model_vert = get_model_vert("vert_highres").load()
#    except KeyError:
#        model_vert = get_model_vert("Vertebra_Highres").load()#

#    BIDS_Global_info.remove_splitting_key("chunk")
#    bids_ds = BIDS_Global_info(datasets=[in_ds], parents=[parent_raw, parent_der], verbose=False)#

#    execution_times = []
#    for name, subject in bids_ds.enumerate_subjects(sort=True):
#        logger = head_logger.add_sub_logger(name=name)
#        q = subject.new_query()
#        q.flatten()
#        q.filter("part", "inphase", required=False)
#        # q.filter("acq", "ax")
#        q.filter("seg", lambda x: x != "manual", required=False)
#        q.filter("lesions", lambda x: x != "manual", required=False)
#        # q.filter("desc", lambda _: False, required=False)
#        q.unflatten()
#        q.filter_format("T2w")
#        q.filter_filetype("nii.gz")
#        families = q.loop_dict(sort=sort, key_addendum=["acq"])
#        for f in families:
#            print(f)
#            try:
#                fid = f.family_id
#                if "T2w_acq-sag" in f:
#                    for t2w in f["T2w_acq-sag"]:
#                        start_time = time.perf_counter()
#                        reduce_nii_size(t2w, t2w.open_nii())
#                        # Call to the pipeline
#                        output_paths, errcode = process_img_nii(
#                            img_ref=t2w,
#                            derivative_name=der,
#                            model_subreg=model_subreg,
#                            model_vert=model_vert,
#                            override_subreg=False,
#                            override_vert=False,
#                            save_debug_data=False,
#                            verbose=False,
#                        )
#                        end_time = time.perf_counter()
#                        execution_time = end_time - start_time
#                        logger.print(f"Inference time is: {execution_time}")
#                        execution_times.append(execution_time)
#                        if errcode == ErrCode.UNKNOWN:
#                            continue
#                        if errcode not in [ErrCode.OK, ErrCode.ALL_DONE]:
#                            logger.print(f"{fid}: Pipeline threw error code {errcode}")
#                if "T2w_acq-ax" not in f or sag_only:
#                    # logger.print(f"{fid}: T2w without part- not found, skip")
#                    continue
#                for t2w in f["T2w_acq-ax"]:
#                    start_time = time.perf_counter()
#                    t2w_nii = t2w.open_nii()
#                    t2w_nii = reduce_nii_size(t2w, t2w_nii)
#                    nii_iso, path_iso = upscale_nii(
#                        nii_org_bids=t2w, parent=iso, override_upscale=override_upscale, batch_size=batch_size, device=device
#                    )
#                    # Call to the pipeline
#                    output_paths, errcode = process_img_nii(
#                        img_ref=BIDS_FILE(path_iso, t2w.dataset),
#                        derivative_name=der,
#                        model_subreg=model_subreg,
#                        model_vert=model_vert,
#                        override_subreg=False,
#                        override_vert=False,
#                        lambda_subreg=filter_segmentation,
#                        save_debug_data=False,
#                        verbose=False,
#                    )
#                    end_time = time.perf_counter()
#                    execution_time = end_time - start_time
#                    logger.print(f"Inference time is: {execution_time}")
#                    execution_times.append(execution_time)
#                    if errcode == ErrCode.UNKNOWN:
#                        continue
#                    if errcode not in [ErrCode.OK, ErrCode.ALL_DONE]:
#                        logger.print(f"{fid}: Pipeline threw error code {errcode}")
#                    # Load Outputs
#                    seg_nii = NII.load(output_paths["out_spine"], seg=True)  # subregion mask
#                    vert_nii = NII.load(output_paths["out_vert"], seg=True)  # subregion mask
#                    poi = calc_centroids_from_subreg_vert(
#                        vert_nii,
#                        seg_nii,
#                        subreg_id=[50, 100, Location.Spinal_Canal_ivd_lvl.value, Location.Spinal_Canal.value],
#                        buffer_file=output_paths["out_ctd"],
#                        save_buffer_file=True,
#                    )
#                    ### RESCALE BACK ###
#                    if resample_back:
#                        output_paths = output_paths_from_input(t2w, der, None)
#                        out_spine = output_paths["out_spine"]
#                        out_vert = output_paths["out_vert"]
#                        out_ctd = output_paths["out_ctd"]
#                        if not out_spine.exists() or override_upscale:
#                            seg = seg_nii.map_labels({Location.Endplate.value: Location.Vertebra_Disc.value})
#                            seg.resample_from_to_(t2w_nii).save(out_spine)
#                        if not out_vert.exists() or override_upscale:
#                            seg = vert_nii.map_labels({i: i % 100 + 100 for i in range(200, 500)})
#                            seg.resample_from_to_(t2w_nii).save(out_vert)
#                        if not out_ctd.exists() or override_upscale:
#                            poi_lr = poi.resample_from_to(t2w_nii)
#                            poi_lr.save(out_ctd)
#                        # vertebra_level = BIDS_FILE(output_paths["out_spine"], t2w.dataset).get_changed_path(info={"seg": "vertebra-level"})#

#                        # poi_lr.extract_subregion(Location.Spinal_Canal_ivd_lvl.value).make_point_cloud_nii(s=4)[0].save(vertebra_level)
#            except Exception:
#                logger.print_error()
#    if len(execution_times) > 0:
#        head_logger.print(
#            f"\nExecution times:\n{execution_times}\nRange:{min(execution_times)}, {max(execution_times)}\nAvg {np.average(execution_times)}"
#        )
